# Тестування durable-func

Notebook зроблена для  управліня durable функціями

In [1]:
import os
import sys
import time
import json
from dotenv import load_dotenv
# читаю конфіг файл
load_dotenv() 
#  додаю шлях для пошуку сервісних модулів
sys.path.append(os.path.abspath(os.path.join('..', '..')))
#  читаю env змінні
HOST_URL = os.getenv("ORCHESTRATOR_HOST")
CONN_STR = os.getenv("STORAGE_CONNECTION_STRING")

# імпортую модулі та створюю сервісні класи
from tests.test_utils import DurableHelper   
helper =  DurableHelper(HOST_URL,CONN_STR ) 
helper.check_helper()



Модуль Helper доступний


## 1. Запуск Orchestrator Function

In [ ]:
PAYLOAD = helper.load_json_file("payload.local.json")
control_urls=helper.start_orchestrator(PAYLOAD)

if control_urls:
    # Збереження URL-адрес у змінних Notebook
    STATUS_URL = control_urls.get('statusQueryGetUri')
    SEND_EVENT_URL = control_urls.get('sendEventPostUri')
    TERMINATE_URL = control_urls.get('terminatePostUri')
    PURGE_URL = control_urls.get('purgeHistoryDeleteUri')
    
    print("✅ Запущено успішно!")
    print(f"ID Інстансу: {control_urls.get('id')}")
    print("\nЗбережені URL-адреси:")
    print(f"Статус: {STATUS_URL}")
    #print ("!!!!!" + response.text)
else:
    print("❌ Помилка при запуску оркестратора!")


## 2. Моніторинг Стану Оркестратора

In [ ]:
import time
import json
if STATUS_URL:
    print("--- Початкова перевірка стану ---")
    # Робимо запит до URL стану
    status_data=helper.check_status(STATUS_URL)
    current_status = status_data.get('runtimeStatus')
    current_status_2 = current_status
    status_data_2 =  status_data 
    print(f"Поточний статус: **{current_status}**")
    # Якщо статус 'Running', можна спробувати почекати
    if current_status == "Running":
        print("Очікування 5 секунд...")
        time.sleep(5)
        # Повторна перевірка
        status_data_2 = helper.check_status(STATUS_URL)
        current_status_2 = status_data_2.get('runtimeStatus')
        print(f"Статус після очікування: **{current_status_2}**")
    if current_status_2 == "Completed":
        # Відображення результату
        output = status_data_2.get('output')
        print("\n✅ Оркестратор завершив роботу. Результат:")
        print(json.dumps(output, indent=2))
    elif current_status_2 == "Failed":
        print("\n❌ Оркестратор завершив роботу з помилкою.")
        print(json.dumps(status_data_2, indent=2))

## 3. Припинити виконання завдання

In [ ]:
# Причина, яка буде записана в логах Orchestrator як причина термінації
REASON = "Terminated by Jupyter Notebook client."
if TERMINATE_URL:
    print(f"Надсилання команди термінації до: {TERMINATE_URL}")
    try:
        status_check = helper.terminate_instance(TERMINATE_URL, REASON, STATUS_URL )
        if status_check:
            if status_check.get('runtimeStatus') == 'Terminated':
                print("✅ Інстанс успішно терміновано.")
            else:
                print(f"❌ Команда термінації надіслана, але поточний статус: {status_check.get('runtimeStatus')}")
        else:
            print("❌ Помилка при спробі термінації: Статус термінації не повернуто!" )
            raise ValueError("Helper не повернув результату!")

    except Exception as e:
        print(f"❌ Помилка при спробі термінації: {e}")

## 4. Очищення історії виконання інстансу

In [ ]:
if PURGE_URL:
    print(f"Видалення історії інстансу: {PURGE_URL}")
    try:
        purge_data = helper.purge_history(PURGE_URL)
        if purge_data:
            # Перевіряємо, чи кількість видалених інстансів > 0
            if purge_data.get('deletedInstanceCount', 0) > 0:
                print(f"✅ Успішно видалено {purge_data.get('deletedInstanceCount')} інстанс(ів).")
            else:
                print(f"Не знайдено даних для очистки")
        else:        
          raise ValueError("Helper не повернув результату!")
    except Exception as e:
        print(f"❌ Помилка при спробі видалення: {e}")

## 5. Вичистити взагалі всю історію всіх інстансів

In [ ]:

PURGE_PAYLOAD = helper.load_json_file("purge-payload.local.json")
print(f"Ця команда видалить усі завершені інстанси (Completed, Failed, Terminated).")
try:
    purge_data = helper.purge_history_all(PURGE_PAYLOAD)
    if purge_data:
        deleted_count = purge_data.get('deletedInstanceCount', 0)
        if deleted_count > 0:
            print(f"✅ Успіх: Видалено {deleted_count} інстанс(ів) через HTTP API.")
        else:
            print("❌ Жоден інстанс, що відповідає критеріям, не знайдено для видалення.")
    else:
        raise ValueError("Helper не повернув результату!")
except Exception as e:
    print(f"❌ Критична помилка при спробі очищення через HTTP: {e}")

## 6. Надсилання зовннішньої події по HTTP для зупинки очікування оркестатора

In [ ]:
if SEND_EVENT_URL:
    print("--- Відправка зовнішньої події ---")
    eventName="PROCESSING_VIDEO_COMPLITED"
    eventPayload = helper.load_json_file("event-payload.local.json")
    eventPayload["instance_id"] = control_urls.get('id')

    try:
        print("Відправляю запит")
        event_data = helper.send_event_http( SEND_EVENT_URL, eventName,  eventPayload)
        if event_data:
            status_code = event_data["status_code"]
            status_text = event_data["status_text"]
            print(f"✅ Response Status Code: {status_code}")
            print(f"💬 Response Status Text: {status_text}")
        else:
            raise ValueError("Helper не повернув результату!")            
    except Exception as e:
        print(f"❌ Критична помилка при відправці зовнішньої події {eventName} через HTTP: {e}")
    

## 7. Надсилання зовннішньої події за допомогою черги для зупинки очікування оркестатора

In [ ]:
q_processed = "video-processed"
eventPayload = helper.load_json_file("event-payload.local.json")
eventPayload["instance_id"] = control_urls.get('id')
try:
    send_result=helper.send_event_queue( CONN_STR, q_processed , eventPayload)
    if send_result:
        print(f"✅ Повідомлення про завершення обробки відправлено до черги { q_processed}.")
    else:
        raise ValueError("Helper не повернув результату!")    
except Exception as e:
    print(f"❌ Помилка при відправці повідомлення до черги: {e}")


